In [16]:
import numpy as np
import torch
import torchvision
import time
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import ReduceLROnPlateau
import nltk
import string

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [7]:
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\Lab716A-PC\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\gutenberg.zip.


True

In [21]:
dataset = nltk.corpus.gutenberg

In [8]:
words = dataset.words()

In [9]:
len(words)

2621613

In [12]:
words[:20]

['[',
 'Emma',
 'by',
 'Jane',
 'Austen',
 '1816',
 ']',
 'VOLUME',
 'I',
 'CHAPTER',
 'I',
 'Emma',
 'Woodhouse',
 ',',
 'handsome',
 ',',
 'clever',
 ',',
 'and',
 'rich']

In [11]:
len(set(words))

51156

In [19]:
len(string.punctuation)

32

In [23]:
n_chars = len(dataset.raw())

In [24]:
n_chars

11793318

In [25]:
subset = dataset.raw()[: n_chars // 100]

In [27]:
char2idx = {}
idx2char = []

In [28]:
for c in subset:
    if c in char2idx:
        continue
    idx2char.append(c)
    char2idx[c] = len(idx2char) - 1

In [34]:
len(idx2char)

72